#### 気体定数$R_v$
$ \displaystyle R_v = \frac{8.314}{0.018}$

In [3]:
const Rv = 8.314/0.018

461.58969804618124

# 水蒸気の状態量に関するモジュール

## 1. 飽和水蒸気圧 $P_{vs}の計算$  
Wexler-Hylandを採用

In [4]:
function cal_Pvs(temp);
    return exp( -5800.22060 / temp + 1.3914993 - 4.8640239E-2 * temp + 4.1764768E-5 * (temp ^ 2.0) - 1.4452093E-8 * (temp ^ 3.0) + 6.5459673 * log(temp) )
end

# 多重ディスパッチによる別実装
cal_Pvs( ;temp ) = cal_Pvs( temp )

cal_Pvs (generic function with 2 methods)

## 2. 水蒸気圧 $P_vの計算$  
### 2.1 温度・相対湿度から求める  
$$P_v = RH * P_{vs} $$

In [5]:
function cal_Pv_ByRH(temp::Float64, rh::Float64)
    return rh * cal_Pvs( temp = temp )
end

cal_Pv_ByRH( ;temp::Float64, rh::Float64 ) = cal_Pv_ByRH( temp, rh )

cal_Pv_ByRH (generic function with 2 methods)

### 2.2 温度・水分化学ポテンシャル水蒸気圧
$$\mu = R_v T log(RH)$$  
$$⇔ P_v = P_v exp \Bigl( \frac{\mu}{R_v T} \Bigl) $$

In [6]:
function cal_Pv_ByMiu(temp::Float64, miu::Float64 );
    rh = exp( miu / Rv / temp )
    return rh * cal_Pvs(temp = temp)
end

cal_Pv_ByMiu(;temp::Float64, miu::Float64 ) = cal_Pv_ByMiu( temp, miu )

cal_Pv_ByMiu (generic function with 2 methods)

## 3. 各種依存性
### 3.1 飽和水蒸気圧の温度微分 
$$\partial P_{vs} / \partial T$$  

# 2022/05/31 出所要確認（おそらく小椋先生のFortranのプログラム）  
# ⇒Wexler Hylandではなさそう

In [7]:
function cal_DPvs( temp );
    DP = 10.795740 * 273.160 / temp / temp - 5.0280 / temp / log(10.0) + 
    ( 1.50475E-4 ) * 8.2969 / 273.16 * log(10.0)* 
    ( 10.0 ^ ( -8.29690 * ( temp / 273.160 - 1.0 ) ) ) + 
    ( 0.42873E-3 ) * 4.769550 * 273.160 / temp / temp * log(10.0) * 
    ( 10.0 ^ ( 4.769550 * ( 1.0 - 273.160 / temp ) ) )
    return cal_Pvs(temp = temp) * DP * log(10.0)
end

cal_DPvs( ;temp ) = cal_DPvs( temp )

cal_DPvs (generic function with 2 methods)

### 3.2 水蒸気圧の温度微分 
$\partial P_{v} / \partial T$  
$P_v = P_{vs} exp(\frac{\mu}{RvT})$

In [8]:
function cal_DPvDT( temp, miu );
    pvs  = cal_Pvs( temp )
    dpvs = cal_DPvs( temp )
    return dpvs * exp( miu / Rv / temp ) - pvs * miu / Rv / temp / temp * exp( miu/ Rv/ temp)
end
cal_DPvDT(; temp, miu ) = cal_DPvDT( temp, miu )

cal_DPvDT (generic function with 2 methods)

In [9]:
function cal_DPvDMiu( temp, miu );
    pvs  = cal_Pvs( temp )
    return pvs / Rv / temp * exp( miu / Rv / temp )
end
cal_DPvDMiu(; temp, miu ) = cal_DPvDMiu( temp, miu )

cal_DPvDMiu (generic function with 2 methods)

### 3.3 相対湿度の水分化学ポテンシャル微分

$$ RH = exp(\frac{\mu}{R_v T})$$  
$$ \frac{\partial RH}{\partial \mu} = \frac{1}{R_v T} exp \Bigl( \frac{\mu}{R_v T} \Bigl) $$

In [10]:
function cal_drh_dmiu( temp::Float64, miu::Float64 )
    return ( 1.0 / Rv / temp ) * exp( miu / Rv / temp )
end
cal_drh_dmiu(; temp::Float64, miu::Float64 ) = cal_drh_dmiu( temp, miu )

cal_drh_dmiu (generic function with 2 methods)

### 3.4 絶対湿度の水蒸気圧微分  

$$ \frac{\partial X}{\partial P_v} =  \frac{0.622P_{atm}}{(P_{atm}-P_v)^2}  $$

In [11]:
function cal_dah_dpv(pv::Float64, patm::Float64=101325.0 )
    return 0.622 * patm / ( ( patm - pv )^2 )
end
cal_dah_dpv(;pv::Float64, patm::Float64=101325.0 ) = cal_dah_dpv(pv, patm )

cal_dah_dpv (generic function with 3 methods)

### 3.5 絶対湿度の相対湿度微分

$$ \frac{\partial X}{\partial h} = \frac{0.622 p_{vs} p_{atm}}{(p_{atm}-p_{vs} h)^2} $$

In [12]:
function cal_dah_drh(temp::Float64, rh::Float64, patm::Float64=101325.0)
    pvs = cal_Pvs(temp)
    return 0.622 * pvs * patm / ( patm - pvs * rh )^2
end
cal_dah_drh(;temp::Float64, rh::Float64, patm::Float64=101325.0) = cal_dah_drh(temp, rh, patm)

cal_dah_drh (generic function with 3 methods)

In [24]:
convertPv2AH( patm = 101325.0, pv = convertRH2Pv( temp =293.15 , rh = 0.6) )

0.00873525355908866

In [26]:
( convertPv2AH( patm = 101325.0, pv = convertRH2Pv( temp =293.15 , rh = 0.7) ) - convertPv2AH( patm = 101325.0, pv = convertRH2Pv( temp =293.15 , rh = 0.6) ) ) /0.1

0.014797852808510065

In [27]:
cal_dah_drh(temp = 303.15, rh = 0.6)

0.027426794896347053

## 4. 水蒸気の各状態量の換算関係

#### 記号

$T$ : Absolute temperature / 絶対温度[K]  
$R_v$ : 水蒸気の気体定数[J kg<sup>-1</sup> K<sup>-1</sup>]  
$RH$ : 相対湿度[-]  
$AH$ : 重量絶対湿度[kg/kg']  
$P_v$ : 水蒸気圧[Pa]  
$P_{vs}$ : 飽和水蒸気圧[Pa]  
$\mu$ : 水分化学ポテンシャル[J/kg] 

### 4.1 RH（相対湿度） - Pv（水蒸気圧）関係  

$ \displaystyle P_v = RH \cdot P_{vs}$

In [14]:
convertRH2Pv( ;temp::Float64, rh::Float64 ) = rh * cal_Pvs( temp = temp )
convertRH2Pv( temp::Float64, rh::Float64 )  = convertRH2Pv( temp = temp, rh = rh )
convertPv2RH( ;temp::Float64, pv::Float64 ) = pv / cal_Pvs( temp = temp )

convertPv2RH (generic function with 1 method)

### 4.2 RH（相対湿度） - Miu（水分化学ポテンシャル）関係  

$ \displaystyle \mu = R_v \cdot T \cdot \log(RH)$

In [ ]:
convertRH2Miu( ;temp::Float64, rh::Float64 )  = Rv * temp * log( rh )
convertMiu2RH( ;temp::Float64, miu::Float64 ) =  exp( miu / Rv / temp )
convertRH2AH( ;temp::Float64, rh::Float64, patm::Float64 = 101325.0 ) =  convertPv2AH( patm = patm, pv = convertRH2Pv( temp, rh ) )

convertMiu2RH (generic function with 1 method)

### 4.3 Pv（水蒸気圧） - Miu（水分化学ポテンシャル）関係  

In [16]:
function convertPv2Miu( ;temp::Float64, pv::Float64 );
    rh = convertPv2RH( temp = temp, pv = pv )
    return convertRH2Miu( temp = temp, rh = rh )
end

function convertMiu2Pv( ;temp::Float64, miu::Float64 );
    rh = convertMiu2RH( temp = temp, miu = miu )
    return convertRH2Pv( temp = temp, rh = rh )
end

convertMiu2Pv (generic function with 1 method)

### 4.4 Pv（水蒸気圧） - AH（絶対湿度）関係  
$$ AH = \frac{0.622 P_v}{P_{atm}-P_v} $$

$$ P_v = \frac{ AH * P_{atm} } { 0.622 + AH } $$

In [17]:
convertPv2AH( ;patm::Float64, pv::Float64 ) = 0.622 * pv / ( patm - pv )
convertAH2Pv( ;patm::Float64, ah::Float64 ) = ah * patm / ( 0.622 + ah )
convertAH2RH( ;temp::Float64, patm::Float64, ah::Float64 ) = convertPv2RH( temp=temp, pv=convertAH2Pv( patm=patm, ah=ah ))

convertAH2RH (generic function with 1 method)

## Example

fig = plt.figure(figsize=(15,4))
ax = fig.add_subplot(1,2,1)

xs = [ n * 0.01 for n in range(27315,37315) ]

for RH in [ 0.2, 0.4, 0.6, 0.8, 1.0 ]:
    ys = [ convertRH2Pv( x, RH ) for x in xs ]
    ax.plot( xs, ys, label = 'RH:' + str(int(RH*100)) + ', %' )

ax.set_xticks([ n * 20 + 273.15 for n in range(0, 6) ])
ax.set_yticks([ n * 10000 for n in range(0, 11) ])
ax.set_xlabel('Absolute temperature, K', fontsize = 15 )
ax.set_ylabel('Vapor pressure, Pa', fontsize = 15 )
plt.legend()
plt.show()

fig = plt.figure(figsize=(15,4))
ax = fig.add_subplot(1,2,1)

xs = [ n * 0.01 for n in range(27315,37315) ]

for RH in [ 0.2, 0.4, 0.6, 0.8, 1.0 ]:
    ys = [ convertRH2Miu( x, RH ) for x in xs ]
    ax.plot( xs, ys, label = 'RH:' + str(int(RH*100)) + ', %' )

ax.set_xticks([ n * 20 + 273.15 for n in range(0, 6) ])
ax.set_yticks([ n * 10000 for n in range(0, 11) ])
ax.set_xlabel('Absolute temperature, K', fontsize = 15 )
ax.set_ylabel('Chemical potential, J/kg', fontsize = 15 )
plt.legend()
plt.show()